# Documentação: Coleta de Repositórios Curtidos (Starred) do GitHub

## Introdução
Este script recupera informações sobre os repositórios curtidos (starred) pelos usuários do GitHub de forma assíncrona. Ele filtra apenas as chaves relevantes e salva os dados processados em arquivos CSV.

## Dependências
Antes de executar o script, instale as bibliotecas necessárias:

```bash
pip install aiohttp nest_asyncio pandas tqdm
```

## Estrutura do Script

### 1. Configuração do Loop Assíncrono
```python
nest_asyncio.apply()
```
Em ambientes como Jupyter Notebook, é necessário aplicar `nest_asyncio` para evitar conflitos com o loop de eventos assíncrono.

### 2. Definição das Chaves Relevantes
```python
chaves_relevantes = ["id", "name", "full_name", "owner_id", "html_url", "description", "fork", "created_at", "updated_at", "pushed_at", "homepage", "size", "stargazers_count", "watchers_count", "language", "forks_count", "open_issues_count", "license_key", "license_name", "topics", "default_branch"]
```
Define quais informações serão extraídas dos repositórios curtidos pelos usuários do GitHub.

### 3. Função `flatten_dict`
```python
def flatten_dict(data, parent_key=""):
```
Essa função achata um dicionário aninhado, transformando listas em strings separadas por vírgulas e mantendo apenas as chaves relevantes.

### 4. Função `fetch_repos_for_user`
```python
async def fetch_repos_for_user(session, user_id, semaphore, timeout=1000):
```
Essa função busca os repositórios curtidos por um usuário no GitHub, tratando a paginação e aplicando a função `flatten_dict` para manter apenas as chaves relevantes.

### 5. Função `get_repository_data_async`
```python
async def get_repository_data_async(user_ids, max_concurrent_requests=10):
```
Essa função gerencia a execução assíncrona para buscar informações dos repositórios curtidos pelos usuários.

### 6. Função `get_repository_data`
```python
def get_repository_data(user_ids):
```
Função wrapper para chamar a versão assíncrona de maneira síncrona.

### 7. Carregamento de IDs e Salvamento dos Dados
```python
users_ids = pd.read_csv(f"{base_path}/users_ids.csv", encoding='utf-8')
users_ids = users_ids['user_id'].to_list()
```
Os IDs dos usuários são lidos de um arquivo CSV, e os dados extraídos são salvos em dois arquivos CSV no caminho definido por `base_path`.

**Fluxo Completo:**
1. Lê a lista de IDs de usuários do GitHub a partir de um CSV.
2. Executa a busca de repositórios curtidos de forma assíncrona.
3. Filtra os dados mantendo apenas as chaves relevantes.
4. Salva os dados extraídos em dois arquivos CSV:
   - `starred_repos_data.csv`: Contém informações dos repositórios curtidos.
   - `starred_repos_users_ids.csv`: Contém os IDs dos usuários e os IDs dos repositórios correspondentes.

### 8. Uso de Variáveis de Ambiente
O script requer duas variáveis de ambiente:
- `GITHUB_TOKEN`: Token de acesso à API do GitHub.
- `DIR_NAME`: Nome do diretório base para armazenamento dos dados processados.

Certifique-se de defini-las antes de executar o script.


# Main

In [1]:
import os
import pandas as pd
import asyncio
import aiohttp
from tqdm.asyncio import tqdm
import nest_asyncio

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Patch para permitir aninhamento de loops (útil em Jupyter Notebook)
nest_asyncio.apply()

# Lista de chaves relevantes para os dados dos repositórios
chaves_relevantes = [
    "id",
    "name",
    "full_name",
    "owner_id",
    "html_url",
    "description",
    "fork",
    "created_at",
    "updated_at",
    "pushed_at",
    "homepage",
    "size",
    "stargazers_count",
    "watchers_count",
    "language",
    "forks_count",
    "open_issues_count",
    "license_key",
    "license_name",
    "topics",
    "default_branch"
]

def flatten_dict(data, parent_key=""):
    """
    Achata um dicionário:
      - Se o valor for um dicionário, adiciona cada subchave como um novo item, concatenando a chave pai e a subchave com um underscore.
      - Se o valor for uma lista, converte a lista para uma string separada por vírgulas.
      - Apenas as chaves (ou chaves compostas para subchaves) que estiverem em 'chaves_relevantes' serão mantidas.
    
    Args:
        data (dict): Dicionário original.
        parent_key (str): Prefixo para as chaves (usado para dicionários aninhados).
    
    Returns:
        dict: Dicionário achatado contendo somente as chaves relevantes.
    """
    flattened = {}
    for key, value in data.items():
        # Cria a nova chave; se houver parent_key, concatena com underscore.
        new_key = f"{parent_key}_{key}" if parent_key else key
        
        if isinstance(value, dict):
            # Para cada subchave, cria uma chave composta e verifica se ela está em chaves_relevantes.
            for sub_key, sub_value in value.items():
                composite_key = f"{new_key}_{sub_key}"
                if composite_key in chaves_relevantes:
                    flattened[composite_key] = sub_value
        elif isinstance(value, list):
            # Se a chave estiver entre as relevantes, converte a lista para string.
            if new_key in chaves_relevantes:
                flattened[new_key] = ", ".join(str(item) for item in value)
        else:
            if new_key in chaves_relevantes:
                flattened[new_key] = value
    return flattened

async def fetch_repos_for_user(session, user_id, semaphore, timeout=1000):
    """
    Busca os dados de repositórios de um usuário, tratando a paginação.
    Após a recuperação, cada dicionário de repositório é achatado pela função flatten_dict,
    mantendo somente as chaves relevantes.
    
    Args:
        session (aiohttp.ClientSession): Sessão HTTP.
        user_id (str ou int): ID do usuário no GitHub.
        semaphore (asyncio.Semaphore): Semáforo para limitar requisições concorrentes.
        timeout (int): Tempo máximo de espera da requisição.
    
    Returns:
        dict: Dicionário com o user_id como chave e a lista de repositórios achatados (somente com as chaves relevantes) como valor.
    """
    url = f"https://api.github.com/user/{user_id}/starred"
    params = {'per_page': 100, 'page': 1}
    repos = []

    async with semaphore:
        while url:
            try:
                async with session.get(url, params=params, timeout=timeout) as response:
                    if response.status == 200:
                        page_data = await response.json()
                        repos.extend(page_data)
                        # Verifica se há próxima página (paginação)
                        if 'next' in response.links:
                            url = response.links['next']['url']
                            params = {}  # A URL já contém os parâmetros necessários
                        else:
                            url = None
                    else:
                        print(f"Erro na requisição para o usuário {user_id}: {response.status}")
                        break
            except asyncio.TimeoutError:
                print(f"Timeout na requisição para o usuário {user_id}")
                break

    # Aplica a função de flatten em cada repositório, mantendo somente as chaves relevantes.
    flattened_repos = [flatten_dict(repo) for repo in repos]
    return {user_id: flattened_repos}

async def get_repository_data_async(user_ids, max_concurrent_requests=10):
    """
    Recupera de forma assíncrona os dados dos repositórios para uma lista de usuários,
    mantendo somente as chaves relevantes definidas em 'chaves_relevantes'.
    
    Args:
        user_ids (list): Lista de IDs de usuários no GitHub.
        max_concurrent_requests (int): Número máximo de requisições HTTP concorrentes.
    
    Returns:
        dict: Dicionário mapeando cada user_id à sua lista de repositórios achatados (filtrados).
    """
    token = os.getenv('GITHUB_TOKEN')
    if not token:
        raise ValueError("A variável de ambiente GITHUB_TOKEN não está definida.")

    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    semaphore = asyncio.Semaphore(max_concurrent_requests)
    results = {}

    async with aiohttp.ClientSession(headers=headers) as session:
        tasks = [fetch_repos_for_user(session, uid, semaphore) for uid in user_ids]
        for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processando Usuários"):
            user_repo_data = await coro
            results.update(user_repo_data)

    return results

def get_repository_data(user_ids):
    """
    Wrapper síncrono para recuperar os dados dos repositórios dos usuários do GitHub.
    
    Args:
        user_ids (list): Lista de IDs de usuários do GitHub.
    
    Returns:
        dict: Dicionário mapeando cada user_id à sua lista de repositórios achatados (filtrados).
    """
    return asyncio.run(get_repository_data_async(user_ids))


In [3]:
dir_name = os.getenv('DIR_NAME')
base_path = f'../data/{dir_name}'
os.makedirs(base_path, exist_ok=True)

users_ids = pd.read_csv(f"{base_path}/users_data.csv", encoding='utf-8')
users_ids = users_ids['id'].to_list()

filename = f"{base_path}/starred_repos_data.csv"
ids_filename = f"{base_path}/starred_repos_users_ids.csv"

stared_data = get_repository_data(users_ids)

Processando Usuários: 100%|██████████| 2029/2029 [02:40<00:00, 12.67it/s]


In [4]:
repos_list = []
ids_list = []
for user_id, repos in stared_data.items():
    for repo in repos:
        repos_list.append(repo)

        repo_id = repo.get("id")
        owner_id = repo.get("owner_id")
        ids_list.append({"user_id": user_id, "repo_id": repo_id, "owner_id": owner_id})


stared_df = pd.DataFrame(repos_list)
ids_df = pd.DataFrame(ids_list)

stared_df.to_csv(filename, index=False, encoding='utf-8')
stared_df.drop_duplicates(subset='id', inplace=True)

ids_df.to_csv(ids_filename, index=False, encoding='utf-8')

print(f'Gravados dados de {len(stared_df)} repositórios curtidos.')

Gravados dados de 17705 repositórios curtidos.
